In [1]:
from pyiron_atomistics import Project

In [2]:
pr = Project('my_dev_stuff')
pr.remove_jobs(silently=True, recursive=True)

  0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
structure = pr.atomistics.structure.bulk('Fe').repeat(4)
structure[0] = 'Cr'
structure[1] = 'Co'

In [11]:
structure.

Signature:
structure.get_potential_energy(
    force_consistent=False,
    apply_constraint=True,
)
Docstring:
Calculate potential energy.

Ask the attached calculator to calculate the potential energy and
apply constraints.  Use *apply_constraint=False* to get the raw
forces.

When supported by the calculator, either the energy extrapolated
to zero Kelvin or the energy consistent with the forces (the free
energy) can be returned.
File:      ~/anaconda3/envs/pyiron_38/lib/python3.8/site-packages/ase/atoms.py
Type:      method


In [4]:
base_job = pr.atomistics.job.Lammps('ref')
base_job.structure = structure.copy()
base_job.potential = base_job.list_potentials()[0]
base_job.calc_minimize()

In [5]:
xy_job = base_job.copy_to(new_job_name='xy')
xy_job.run()
E_xy = xy_job.output.energy_pot[-1]

The job xy was saved and received the ID: 166


In [6]:
x_job = base_job.copy_to(new_job_name='x')
x_job.structure[1] = 'Fe'
x_job.run()
E_x = x_job.output.energy_pot[-1]

The job x was saved and received the ID: 167


In [7]:
y_job = base_job.copy_to(new_job_name='y')
y_job.structure[0] = 'Fe'
y_job.run()
E_y = y_job.output.energy_pot[-1]

The job y was saved and received the ID: 168


In [8]:
bulk_job = base_job.copy_to(new_job_name='bulk')
bulk_job.structure[[0,1]] = 'Fe'
bulk_job.run()
E_bulk = bulk_job.output.energy_pot[-1]

The job bulk was saved and received the ID: 169


In [9]:
E_bind = (E_xy + E_bulk) - (E_x + E_y)

In [10]:
E_bind

0.03351011349991495